# LangGraph with Memory

In this notebook, we demonstrate how to integrate memory into a LangGraph-based agent. This allows the agent to maintain context across steps and reuse past information.

In [ ]:
# !pip install langgraph langchain openai

In [ ]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

# Initialize components
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory(return_messages=True)

## Define Nodes
- The `contextual_executor` will access memory before answering.
- The `recorder` will store interactions in memory.

In [ ]:
# Executor with memory usage
def contextual_executor(state):
    memory.load_memory_variables({})  # Load memory
    user_input = state.get("input", "")
    response = llm.predict(f"Context: {memory.buffer}\nUser: {user_input}")
    return {"input": user_input, "response": response}

In [ ]:
# Recorder stores the new interaction
def recorder(state):
    memory.save_context({"input": state["input"]}, {"output": state["response"]})
    return state

## Build the graph

In [ ]:
graph = StateGraph()
graph.add_node("contextual_executor", contextual_executor)
graph.add_node("recorder", recorder)

graph.set_entry_point("contextual_executor")
graph.add_edge("contextual_executor", "recorder")
graph.add_edge("recorder", END)

app = graph.compile()

## Run the agent

In [ ]:
# Example 1
app.invoke({"input": "What is 2 + 2?"})

In [ ]:
# Example 2: builds on memory
app.invoke({"input": "And what if we subtract 1 from that?"})